In [2]:
%%manim -ql -v WARNING MatrixLatticeScene

from manim import *  # Make sure all constants are imported

class MatrixLatticeScene(Scene):
    def construct(self):
        # Set white background
        self.camera.background_color = WHITE
        
        # Define matrix entries for the different levels
        root_entries = [["a", "b"], ["c", "d"]]
        second_level_entries = [
            [["a", "b"], ["c", "d"]] for _ in range(8)
        ]
        third_level_entries = [
            [["a", "b"], ["c", "d"]] for _ in range(8)
        ]
        bottom_entries = [["a", "b"], ["c", "d"]]
        
        # Create root matrix
        root_matrix = self.draw_matrix_meet_product(
            root_entries, 
            meet_positions=[(0, 1)], # Highlight this as a meet position
            cell_w=0.7, 
            cell_h=0.6
        ).scale(0.8)
        root_matrix.move_to(UP * 3)
        
        # Create second level matrices
        second_level = VGroup()
        for i in range(8):
            # Alternate which cells are meet positions
            meet_pos = [(0, 1)] if i % 2 == 0 else [(1, 0)]
            matrix = self.draw_matrix_meet_product(
                second_level_entries[i],
                meet_positions=meet_pos,
                cell_w=0.7, 
                cell_h=0.6
            ).scale(0.7)
            second_level.add(matrix)
        
        second_level.arrange(RIGHT, buff=0.2)
        second_level.next_to(root_matrix, DOWN, buff=1.2)
        
        # Create third level matrices
        third_level = VGroup()
        for i in range(8):
            # Some will be meet products
            meet_pos = [(0, 0)] if i % 3 == 0 else None
            matrix = self.draw_matrix_meet_product(
                third_level_entries[i],
                meet_positions=meet_pos,
                cell_w=0.7, 
                cell_h=0.6
            ).scale(0.7)
            third_level.add(matrix)
        
        third_level.arrange(RIGHT, buff=0.2)
        third_level.next_to(second_level, DOWN, buff=1.2)
        
        # Create bottom matrix
        bottom_matrix = self.draw_matrix_meet_product(
            bottom_entries,
            meet_positions=[(0, 0), (1, 1)], # Both diagonal cells are meets
            cell_w=0.7, 
            cell_h=0.6
        ).scale(0.8)
        bottom_matrix.next_to(third_level, DOWN, buff=1.2)
        
        # Create connecting lines
        lines = VGroup()
        
        # Connect root to second level
        for matrix in second_level:
            line = Line(
                root_matrix.get_bottom(), 
                matrix.get_top(),
                color=BLUE_C,
                stroke_width=1
            )
            lines.add(line)
        
        # Connect second level to third level
        for i, start_matrix in enumerate(second_level):
            # Connect each matrix to corresponding ones in third level
            # Spreading connections more widely for better visibility
            indices = range(max(0, i-1), min(len(third_level), i+2))
            for j in indices:
                line = Line(
                    start_matrix.get_bottom(),
                    third_level[j].get_top(),
                    color=BLUE_C,
                    stroke_width=1
                )
                lines.add(line)
        
        # Connect third level to bottom
        for matrix in third_level:
            line = Line(
                matrix.get_bottom(),
                bottom_matrix.get_top(),
                color=BLUE_C,
                stroke_width=1
            )
            lines.add(line)
        
        # Add all elements to the scene
        self.add(lines, root_matrix, second_level, third_level, bottom_matrix)
        
        # Scale everything to fit the screen better
        everything = VGroup(lines, root_matrix, second_level, third_level, bottom_matrix)
        
        # Fix for FRAME_HEIGHT error - use config.frame_height instead
        everything.scale_to_fit_height(config.frame_height - 1)  # Using config
        everything.move_to(ORIGIN)
        
    # Include your draw_matrix_meet_product function here
    def draw_matrix_meet_product(
            self, entries, meet_positions=None, *, cell_w: float = 2.7, cell_h: float = 0.9
        ) -> VGroup:
            """
            Draw a matrix with optional meet vector product indicators.

            Args:
                entries: 2D array of cell contents
                meet_positions: List of tuples (r, c) indicating cells that represent meet products
                cell_w: Width of each cell
                cell_h: Height of each cell

            Returns:
                VGroup containing the matrix
            """
            rows, cols = len(entries), len(entries[0])
            matrix = VGroup()
            top_left = UP * ((rows - 1) * cell_h / 2) + LEFT * ((cols - 1) * cell_w / 2)

            # Create the base matrix first
            cells = {}  # Store cell references for later annotation

            for r in range(rows):
                for c in range(cols):
                    cell_text = Text(entries[r][c], font_size=19, color=BLACK)

                    # Default cell background
                    cell_bg = Rectangle(
                        width=cell_w,
                        height=cell_h,
                        stroke_color=BLACK,
                        stroke_width=2,
                        fill_color=WHITE,
                        fill_opacity=0.8,
                    )

                    # Group text and background
                    cell_group = VGroup(cell_bg, cell_text).move_to(
                        top_left + RIGHT * c * cell_w + DOWN * r * cell_h
                    )

                    # Store reference to the cell
                    cells[(r, c)] = cell_group

                    # Add to the matrix
                    matrix.add(cell_group)

            # If meet positions are provided, add visual indicators
            if meet_positions:
                for r, c in meet_positions:
                    if (r, c) in cells:
                        # Get the cell to modify
                        cell = cells[(r, c)]

                        # Add a special background to indicate this is a meet product
                        meet_indicator_bg = RoundedRectangle(
                            width=cell_w - 0.1,
                            height=cell_h - 0.1,
                            stroke_color=BLUE,
                            stroke_width=3,
                            fill_color=BLUE_A,
                            fill_opacity=0.3,
                            corner_radius=0.1
                        )

                        # Add small meet symbol (∧) indicator in corner
                        meet_symbol = Text("∧", font_size=16, color=BLUE_E).move_to(
                            cell.get_center() + UL * (cell_w/2 - 0.2)
                        )

                        # Add these to the cell group
                        cell.add(meet_indicator_bg, meet_symbol)
            return matrix

UsageError: Cell magic `%%manim` not found.


In [2]:
import pytest
from brancharchitect.partition import Partition
from brancharchitect.partition_set import PartitionSet, count_full_overlaps

def test_count_full_overlaps_basic():
    # PartitionSet: {[1,2], [2,3], [1,2,3]}
    ps = PartitionSet([
        Partition((1, 2), {}),
        Partition((2, 3), {}),
        Partition((1, 2, 3), {}),
    ])
    target = Partition((1, 2), {})
    # [1,2] is fully contained in itself and in [1,2,3]
    assert count_full_overlaps(target, ps) == 2

def test_count_full_overlaps_no_overlap():
    ps = PartitionSet([
        Partition((4, 5), {}),
        Partition((6,), {}),
    ])
    target = Partition((1, 2), {})
    # No partition contains [1,2]
    assert count_full_overlaps(target, ps) == 0

def test_count_full_overlaps_all():
    ps = PartitionSet([
        Partition((1, 2), {}),
        Partition((1, 2), {}),
        Partition((1, 2), {}),
    ])
    target = Partition((1, 2), {})
    # All partitions are identical to target
    assert count_full_overlaps(target, ps) == 3

def test_count_full_overlaps_partial_overlap():
    ps = PartitionSet([
        Partition((1, 2), {}),
        Partition((2, 3), {}),
        Partition((1, 3), {}),
    ])
    target = Partition((1, 2), {})
    # Only [1,2] contains [1,2]
    assert count_full_overlaps(target, ps) == 1
    
test_count_full_overlaps_basic()
test_count_full_overlaps_no_overlap()